Using the library from Hamamatsu

In [ ]:
import asyncio
 
 
async def func1():
    print("Function 1 started..")
    await asyncio.sleep(2)
    print("Function 1 Ended")
 
 
async def func2():
    print("Function 2 started..")
    await asyncio.sleep(3)
    print("Function 2 Ended")
 
 
async def func3():
    print("Function 3 started..")
    await asyncio.sleep(1)
    print("Function 3 Ended")
 
 
async def main():
    L = await asyncio.gather(
        func1(),
        func2(),
        func3(),
    )
    print("Main Ended..")
 
 
asyncio.run(main())
 

Using Asyncio

In [1]:
import asyncio, time
async def loop():
    print("Printing numbers")
    for i in range(0,100):
        print(i)
        time.sleep(1)

In [2]:
import asyncio
async def coro_func():
    print("Hello, asyncio!")

# to run the code defined in a coroutine function, you need to await it...
# However, you can’t await it in the same way as you iterate a generator. A coroutine can only be awaited inside another coroutine defined by the async def syntax:
async def main():
    print("In the entrypoint coroutine.")
    await coro_func()

# How to run the main() coroutine?
# For the top-level entry point coroutine function, which is normally named as main(), we need to use asyncio.run() to run it:
# asyncio.run(main())

# In your case, jupyter (IPython ≥ 7.0) is already running an event loop.
# Therefore you don't need to start the event loop yourself and can instead call await main(url) directly, even if your code lies outside any asynchronous function.
await main()

In the entrypoint coroutine.
Hello, asyncio!


In [3]:
# Until now a single corountine.. Now multiple coroutines...
# INCORRECT AWAITING

import asyncio
from datetime import datetime

async def async_sleep(num):
    print(f"Sleeping {num} seconds.")
    await asyncio.sleep(num)    # asyncio.sleep() simulates IO blocking time

async def main():
    start = datetime.now()

    for i in range(1, 4):
        await async_sleep(i)
    
    duration = datetime.now() - start
    print(f"Took {duration.total_seconds():.2f} seconds.")

await main()

Sleeping 1 seconds.
Sleeping 2 seconds.
Sleeping 3 seconds.
Took 6.03 seconds.


In [10]:
# To achieve concurrency, we need to run multiple coroutines with the async.gather() function - to run multiple awaitables concurrently

import asyncio
from datetime import datetime

async def async_sleep(num):
    print(f"Sleeping {num} seconds.")
    await asyncio.sleep(num)    # simulating TO blocking time
    print(f"Done {num}.")

async def main():
    start = datetime.now()

    coro_objs = []
    for i in [1,3,5,10]:
        coro_objs.append(async_sleep(i))
    
    # await them together using gather(): unpack them into gather()
    await asyncio.gather(*coro_objs)    # the starred expression is a feature that allows for the unpacking of elements from iterable like lists

    duration = datetime.now() - start
    print(f"Took {duration.total_seconds():.2f} seconds.")

# asyncio.run(main())
await main()

Sleeping 1 seconds.
Sleeping 3 seconds.
Sleeping 5 seconds.
Sleeping 10 seconds.
Done 1.
Done 3.
Done 5.
Done 10.
Took 10.00 seconds.


In [5]:
import asyncio
async def async_sleep(num):
    print(f"Sleeping {num} seconds.")
    await asyncio.sleep(num)    # simulating IO blocking time

coro_objs = []
for i in range(1,4):
    coro_objs.append(async_sleep(i))

type(coro_objs)

list

In [7]:
aw = asyncio.gather(*coro_objs)
type(aw)

asyncio.tasks._GatheringFuture

Generator: can be used in multiprocessing to display the captured images live or acquired voltage live.

The main adv being the two processes (acquisition and plotting) can run at different speeds

In [15]:
# Generator is defined as a normal function
# the 'return' is replaced by 'yield'
def gen_func():
    for i in range(0,10):
        yield i     # this makes a generator

generator_obj = gen_func()
type(generator_obj)

generator

In [ ]:
# to get output from a generator, call next(<generator object>) to iterate
next(generator_obj)

In [11]:
from PyQt5 import QtCore, QtGui, QtWidgets
import matplotlib.pyplot as plt

"Using Asyncio for camera acquisition"

In [ ]:
import asyncio, matplotlib.pyplot as plt, sys, time
from datetime import datetime
from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_MainWindow(QtWidgets.QWidget):
    def setupUi(self, MainWindow):
        def __init__(self):
            super().__init__()  # Call parent class constructor
            self.setupUi(self)  # Call setupUi after object creation
            self.show()  # Explicitly show the window (if __init__ exists)

        MainWindow.resize(422, 255)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
 
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(160, 130, 93, 28))
 
        # For displaying confirmation message along with user's info.
        self.label = QtWidgets.QLabel(self.centralwidget)   
        self.label.setGeometry(QtCore.QRect(170, 40, 201, 111))
 
        # Keeping the text of label empty initially.      
        self.label.setText("")    

        MainWindow.setCentralWidget(self.centralwidget)
        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
 
    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButton.setText(_translate("MainWindow", "Proceed"))
        self.pushButton.clicked.connect(self.takeinputs)

    async def takeinputs(self):
        exp_time=10;
        while True:
            exp_time, done2 = QtWidgets.QInputDialog.getInt(self, 'Exposure', 'Enter exposure time [ms]:',value=exp_time,min=1,max=10000)
            if not done2:
                plt.close("Set Exposure")
                QtCore.QCoreApplication.quit()
                break

async def loop():
    print("Running task (loop)...")
    for i in range(0,100):
        print(i)
        # asyncio.sleep(1)

async def main():
    start = datetime.now()

    print("Exposure set control active..")
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()

    # Call setupUi directly before gather
    ui.setupUi(MainWindow)

    coro_objs = [loop()]  # Only include the actual coroutine (loop)
    await asyncio.gather(*coro_objs)  # Gather only the coroutine

    app.exec_()
    app.quit()
    duration = datetime.now() - start
    print(f"Took {duration.total_seconds():.2f} seconds.")

# asyncio.run(main())
await main()


In [2]:
import asyncio, matplotlib.pyplot as plt, sys, time
from datetime import datetime
from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_MainWindow(QtWidgets.QWidget):
    def setupUi(self, MainWindow):
        MainWindow.resize(422, 255)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
 
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(160, 130, 93, 28))
 
        # For displaying confirmation message along with user's info.
        self.label = QtWidgets.QLabel(self.centralwidget)   
        self.label.setGeometry(QtCore.QRect(170, 40, 201, 111))
 
        # Keeping the text of label empty initially.      
        self.label.setText("")    

        MainWindow.setCentralWidget(self.centralwidget)
        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
 
    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButton.setText(_translate("MainWindow", "Proceed"))
        self.pushButton.clicked.connect(self.takeinputs)

    def takeinputs(self):
        exp_time=10;
        while True:
            exp_time, done2 = QtWidgets.QInputDialog.getInt(self, 'Exposure', 'Enter exposure time [ms]:',value=exp_time,min=1,max=10000)
            if not done2:
                QtCore.QCoreApplication.quit()
                break

def set_exposure():
    print("Exposure set control active...")
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    app.exec_()
    app.quit()
    print("Exposure set...")

if __name__ == "__main__":
    set_exposure()

Exposure set control active...
Exposure set...


In [7]:
import numpy as np
from skimage import io
%matplotlib qt
frame = np.random.randn(100,100)
io.imshow(frame)

In [8]:
import threading

class KeyboardThread(threading.Thread):

    def __init__(self, input_cbk = None, name='keyboard-input-thread'):
        self.input_cbk = input_cbk
        super(KeyboardThread, self).__init__(name=name, daemon=True)
        self.start()

    def run(self):
        while True:
            self.input_cbk(input()) #waits to get input + Return

showcounter = 0 #something to demonstrate the change

def my_callback(inp):
    #evaluate the keyboard input
    print('You Entered:', inp, ' Counter is at:', showcounter)

#start the Keyboard thread
kthread = KeyboardThread(my_callback)

while True:
    #the normal program executes without blocking. here just counting up
    showcounter += 1



You Entered: 8  Counter is at: 95220582
You Entered: 6  Counter is at: 137064590
You Entered: 47  Counter is at: 186478798
You Entered: 3  Counter is at: 215579857
You Entered:   Counter is at: 232002039
You Entered:   Counter is at: 291814248
You Entered:   Counter is at: 320165043
You Entered: q  Counter is at: 600816208
You Entered:   Counter is at: 645697852
You Entered:   Counter is at: 661882413
You Entered:   Counter is at: 1244285126
You Entered:   Counter is at: 1257992926
You Entered:   Counter is at: 1277629345
You Entered:   Counter is at: 1288869704
You Entered:    Counter is at: 1316089586
You Entered:   Counter is at: 1324296645
You Entered:   Counter is at: 1331421702
You Entered:   Counter is at: 1337229053
You Entered:   Counter is at: 1341496825
You Entered: 

In [1]:
"""
https://stackoverflow.com/questions/5404068/how-to-read-keyboard-input/53344690#53344690

read_keyboard_input.py

Gabriel Staples
www.ElectricRCAircraftGuy.com
14 Nov. 2018

References:
- https://pyserial.readthedocs.io/en/latest/pyserial_api.html
- *****https://www.tutorialspoint.com/python/python_multithreading.htm
- *****https://en.wikibooks.org/wiki/Python_Programming/Threading
- https://stackoverflow.com/questions/1607612/python-how-do-i-make-a-subclass-from-a-superclass
- https://docs.python.org/3/library/queue.html
- https://docs.python.org/3.7/library/threading.html

To install PySerial: `sudo python3 -m pip install pyserial`

To run this program: `python3 this_filename.py`

"""

import threading
import queue
import time

def read_kbd_input(inputQueue):
    print('Ready for keyboard input:')
    while (True):
        # Receive keyboard input from user.
        input_str = input()
        
        # Enqueue this input string.
        # Note: Lock not required here since we are only calling a single Queue method, not a sequence of them 
        # which would otherwise need to be treated as one atomic operation.
        inputQueue.put(input_str)

def main():

    EXIT_COMMAND = "exit" # Command to exit this program

    # The following threading lock is required only if you need to enforce atomic access to a chunk of multiple queue
    # method calls in a row.  Use this if you have such a need, as follows:
    # 1. Pass queueLock as an input parameter to whichever function requires it.
    # 2. Call queueLock.acquire() to obtain the lock.
    # 3. Do your series of queue calls which need to be treated as one big atomic operation, such as calling
    # inputQueue.qsize(), followed by inputQueue.put(), for example.
    # 4. Call queueLock.release() to release the lock.
    # queueLock = threading.Lock() 

    #Keyboard input queue to pass data from the thread reading the keyboard inputs to the main thread.
    inputQueue = queue.Queue()

    # Create & start a thread to read keyboard inputs.
    # Set daemon to True to auto-kill this thread when all other non-daemonic threads are exited. This is desired since
    # this thread has no cleanup to do, which would otherwise require a more graceful approach to clean up then exit.
    inputThread = threading.Thread(target=read_kbd_input, args=(inputQueue,), daemon=True)
    inputThread.start()

    # Main loop
    while (True):

        # Read keyboard inputs
        # Note: if this queue were being read in multiple places we would need to use the queueLock above to ensure
        # multi-method-call atomic access. Since this is the only place we are removing from the queue, however, in this
        # example program, no locks are required.
        if (inputQueue.qsize() > 0):
            input_str = inputQueue.get()
            print("input_str = {}".format(input_str))

            if (input_str == EXIT_COMMAND):
                print("Exiting serial terminal.")
                break # exit the while loop
            
            # Insert your code here to do whatever you want with the input_str.

        # The rest of your program goes here.

        # Sleep for a short time to prevent this thread from sucking up all of your CPU resources on your PC.
        time.sleep(0.01) 
    
    print("End.")

# If you run this Python file directly (ex: via `python3 this_filename.py`), do the following:
if (__name__ == '__main__'): 
    main()

Ready for keyboard input:
input_str = e
input_str = j
input_str = 
input_str = 
input_str = 
input_str = l
input_str = 
input_str = 
input_str = exit
Exiting serial terminal.
End.


In [36]:
import dcam

dcam.Dcamapi.init()     # initialize DCAM-API

True

In [15]:
dcam.Dcamapi.get_devicecount()

1

In [16]:
hdcam = dcam.Dcam(0)

In [28]:
hdcam.dev_getstring(dcam.DCAM_IDSTR.MODEL)

'C13440-20CU'

In [21]:
hdcam.dev_getstring(dcam.DCAM_IDSTR.CAMERAID)

'S/N: 303200'

In [27]:
hdcam.dev_close()

True

In [29]:
hdcam.dev_open()

True

In [30]:
idprop = hdcam.prop_getnextid(0)

In [33]:
propname = hdcam.prop_getname(idprop)
propname

'SENSOR MODE'

In [34]:
hdcam.Dcampi.uninit()

AttributeError: module 'dcam' has no attribute 'Dcampi'

#### Using dcamcon

In [5]:
import dcamcon
dcamcon.dcamcon_init()
hdcamcon = dcamcon.dcamcon_choose_and_open()       # call this hdcamcon (dcamcon handle) instead of hdcam (dcam handle in doc)
print(type(hdcamcon))

Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
<class 'dcamcon.Dcamcon'>


In [7]:
hdcamcon.dcam.dev_getstring(dcamcon.DCAM_IDSTR.CAMERA_SERIESNAME)

'ORCA-Flash4.0 V3'

In [8]:
dcamcon.dcamcon_list[0].device_title
# hdcam is the dcamcon.dcamcon_list[0] done inside dcamcon_choose_and_open()

'#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L'

In [10]:
# get the device strings: TARGET DCAM device
print(hdcamcon.dcam.dev_getstring(idstr=dcamcon.DCAM_IDSTR.CAMERA_SERIESNAME))
print(hdcamcon.dcam.dev_getstring(idstr=dcamcon.DCAM_IDSTR.MODEL))

ORCA-Flash4.0 V3
C13440-20C


In [12]:
# get the value of DCAM_IDPROP using the DCAMCON functions (not DCAM directly)
print(hdcamcon.get_propertyvalue(propid=dcamcon.DCAM_IDPROP.TRIGGER_MODE))
print(hdcamcon.get_propertyvalue(propid=dcamcon.DCAM_IDPROP.EXPOSURETIME))

1.0
0.009997714285714285


##### Setting ROI!!

In [19]:
print(hdcamcon.setget_propertyvalue(propid=dcamcon.DCAM_IDPROP.SUBARRAYMODE,val=dcamcon.DCAMPROP.MODE.ON))

2.0


###### setting subarray is not straightforward...

In [21]:
print(hdcamcon.set_propertyvalue(propid=dcamcon.DCAM_IDPROP.SUBARRAYHPOS,val=100))

-NG: Dcam.prop_setvalue(SUBARRAYHPOS, 100) failed with error INVALIDSUBARRAY
False


In [41]:
propattr_offset = hdcamcon.dcam.prop_getattr(dcamcon.DCAM_IDPROP.SUBARRAYHPOS)

In [45]:
propattr_size = hdcamcon.dcam.prop_getattr(dcamcon.DCAM_IDPROP.SUBARRAYHSIZE)

In [51]:
is_offset_available = (propattr_offset.attribute & dcamcon.DCAM_PROP.ATTR.EFFECTIVE and
                                       propattr_offset.attribute & dcamcon.DCAM_PROP.ATTR.WRITABLE and
                                       propattr_offset.valuemin != propattr_offset.valuemax)

True

In [53]:
is_size_available = (propattr_size.attribute & dcamcon.DCAM_PROP.ATTR.EFFECTIVE and
                                     propattr_size.attribute & dcamcon.DCAM_PROP.ATTR.WRITABLE and
                                     propattr_size.valuemin != propattr_size.valuemax)

In [25]:
# using set_propertyvalue(): set a value of DCAM_IDPROP using the DCAMCON functions (not DCAM directly)
hdcamcon.set_propertyvalue(propid=dcamcon.DCAM_IDPROP.EXPOSURETIME, val=0.1)

True

In [29]:
# using setget_propertyvalue(): set a value of DCAM_IDPROP using the DCAMCON functions (not DCAM directly)
hdcamcon.setget_propertyvalue(propid=dcamcon.DCAM_IDPROP.EXPOSURETIME, val=0.05)

0.05000806015037594

In [6]:
# when closing camera software by calling dcamcon_uninit()
# the function closes the camera by calling my.close() (calls self.dcam.dev_close())
# the function also calls Dcamapi.uninit()

dcamcon.dcamcon_uninit()

DCAM-API uninit'd..


In [66]:
if not hdcam.allocbuffer(100):
    print("Buffer not allocated..")
    

-NG: Dcam.buf_alloc(100) failed with error NOTSTABLE
Buffer not allocated..


In [42]:
hdcam.get_propertyvalue(propid=dcamcon.DCAMCAP_STATUS)
# DCAMCAP_STATUS is not a property.. hence this call is not valid..

AttributeError: value

In [73]:
hdcam.get_propertyvalue(dcamcon.DCAM_IDPROP.TRIGGERSOURCE)

1.0

In [72]:
# the way to get the correct 'enum's for a specific property..
print(list(map(int, dcamcon.DCAMPROP.TRIGGERSOURCE)))
print(list(dcamcon.DCAMPROP.TRIGGERSOURCE))
dcamcon.DCAMPROP.TRIGGERSOURCE.SOFTWARE

[1, 2, 3, 4]
[<TRIGGERSOURCE.INTERNAL: 1>, <TRIGGERSOURCE.EXTERNAL: 2>, <TRIGGERSOURCE.SOFTWARE: 3>, <TRIGGERSOURCE.MASTERPULSE: 4>]


<TRIGGERSOURCE.SOFTWARE: 3>

In [71]:
hdcam

In [75]:
import cv2, screeninfo
if not hdcam.startcapture(is_sequence=False):
    # dcamcon.allocbuffer() should have succeeded
    hdcam.releasebuffer()
    # return 

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """

    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = screeninfo.get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

timeout_millisec=500
global cv_window_status
cv_window_status = 1
while cv_window_status >= 0:
    res = hdcam.wait_capevent_frameready(timeout_millisec)
    # if res is not True:
    #     # frame does not come
    #     if res != DCAMERR.TIMEOUT:
    #         print('-NG: Dcam.wait_event() failed with error {}'.format(res))
    #         break

    #     # TIMEOUT error happens
    #     timeout_happened += 1
    #     if timeout_happened == 1:
    #         print('Waiting for a frame to arrive.', end='')
    #         if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
    #             print(' Check your trigger source.', end ='')
    #         else:
    #             print(' Check your <timeout_millisec> calculation in the code.', end='')
    #         print(' Press Ctrl+C to abort.')
    #     else:
    #         print('.')
    #         if timeout_happened > 5:
    #             timeout_happened = 0
        
    #     continue

    # wait_capevent_frameready() succeeded
    lastdata = hdcam.get_lastframedata()
    if lastdata is not False:
        show_framedata('Capture', lastdata)
    
    timeout_happened = 0

# End live
cv2.destroyAllWindows()

In [ ]:
import cv2
if dcam.cap_start() is not False:
    timeout_milisec = 100
    cv2.namedWindow('test', cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow('test', 700, 700)
    iWindowStatus = 0
    while iWindowStatus >= 0:
        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
            data = dcam.buf_getlastframedata()
            
        else:
            dcamerr = dcam.lasterr()
            if dcamerr.is_timeout():
                print('===: timeout')
            else:
                print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                break

        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q') or key == 32:  # if 'q' was pressed with the live window, close it
            break
    cv2.destroyAllWindows()
    dcam.cap_stop()

In [2]:
import numpy as np

print(np.__file__)

c:\ProgramData\Anaconda3\lib\site-packages\numpy\__init__.py


In [ ]:
import threading
import time

# Dummy function to simulate camera behavior
def camera_acquire_frame(trigger_event, num_frames):
    frames_acquired = 0
    while frames_acquired < num_frames:
        trigger_event.wait()  # Wait for the trigger event to be set
        print(f"Camera: Acquiring frame {frames_acquired + 1}")
        frames_acquired += 1
        trigger_event.clear()  # Reset the event to wait for the next pulse

# Main function to handle pulse generation and camera acquisition
def main():
    num_frames = 10
    pulse_interval = 0.5  # Adjust as needed

    # Event object to signal between threads
    trigger_event = threading.Event()

    # Create and start the camera thread
    camera_thread = threading.Thread(target=camera_acquire_frame, args=(trigger_event, num_frames))
    camera_thread.start()

    # Main thread handling pulse generation
    for pulse in range(num_frames):
        time.sleep(pulse_interval)  # Simulate time between pulses
        print(f"Pulse Generator (Main Thread): Sending pulse {pulse + 1}")
        trigger_event.set()  # Set the event to trigger the camera
        trigger_event.clear()

    # Wait for the camera thread to complete
    camera_thread.join()

    print("Acquisition complete")

if __name__ == "__main__":
    main()
